# Transformações no Dataset

## Sobre o Dataset

Esse dataset anonimizado é um grafo de transações coletado da blockchain do Bitcoin. Um nó no grafo representa uma transação, uma aresta pode ser vista como um fluxo de bitcoins entre uma transação e outra. Cada nó possui 166 atributos e foi rotulado como tendo sido criado por uma entidade “lícita”, “ilícita” ou “desconhecida”.


__Nós e Arestas__

O grafo é composto de 203.769 nós e 234.355 arestas. Dois por cento (4.545) dos nós estão rotulados como class1 (ilícito). Vinte e um por cento (42,019) estão nomeados como class2 (lícito). As transações remanescentes não estão rotuladas como lícitas ou ilícitas.

__Atributos__

Existem 166 atributos associados a cada nó. Devido a problemas de propriedade intelectual, não é possível prover uma descrição exata de todos os atributos na base de dados. Há um passo de tempo associado a cada nó, representando a medida de tempo em que a transação foi emitida na rede do Botcoin. Cada passo de tempo, indo de 1 a 49, estão espaçados dentro de um intervalo médio de duas semanas. Cada passo de tempo tempo contém um único componente conectado de transações que apareceram na blockchain em menos de três horas entre uma e outra; não há arestas conectando passos de tempo diferentes
Os primeiros 94 atributos representam a informação local sobre a transação – incluindo o tempo descrito acima, o número de entradas/saídas, a taxa de transação, volume de saída e números agregados, como o BTC médio recebido (gasto) pelas entradas/saídas e o número médio de transações de entrada (saída) associadas às entradas/saídas. Os 72 atributos restantes são atributos agregados, obtidos usando informações da transação um salto no tempo para trás/frente do nó central - tendo o máximo, mínimo, desvio padrão e coeficientes de correlação das transações vizinhas para os mesmos dados de informação (número de entradas/saídas, taxa de transação, etc.).

__License__

This data set is distributed under the Creative CommonsAttribution-NonCommercial-NoDerivatives License 4.0 International.

__Fonte__: https://www.kaggle.com/ellipticco/elliptic-data-set

## Importando as Bibliotecas

Esse _notebook_ converte o conjunto de dados para o padrão PyTorch Geometric, para servir de entrada para a rede neural, implementada nessa mesma biblioteca.

In [10]:
import os
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data

## Carregar os Dados

In [2]:
path = 'elliptic_bitcoin_dataset/'

elliptic_txs_classes = pd.read_csv(path + 'elliptic_txs_classes.csv')
elliptic_txs_edgelist = pd.read_csv(path + 'elliptic_txs_edgelist.csv')
elliptic_txs_features = pd.read_csv(path + 'elliptic_txs_features.csv', header = None)

Removendo a classe "unknown"

In [3]:
filter = [elliptic_txs_classes != 'unknown']

In [4]:
filter = filter[0]['class']

## Processamento de dados

### Função para ajustar a lista de arestas

In [5]:
def feedEdgeList(row):
    row['classv1'] = elliptic_txs_classes[elliptic_txs_classes['txId'] == row['txId1']]['class'].values[0]
    row['classv2'] = elliptic_txs_classes[elliptic_txs_classes['txId'] == row['txId2']]['class'].values[0]
    row['graph'] = elliptic_txs_features[elliptic_txs_features[0] == row['txId1']][1].values[0]
    return row

elliptic_txs_edgelist = elliptic_txs_edgelist.apply(feedEdgeList, axis=1)

### Processando os dados

In [6]:
def updateVerticeNumber(row, dt):
    row['txId1'] = dt[row['txId1']]
    row['txId2'] = dt[row['txId2']]
    return row

datas = []

for i in range(1, 50):
    features = elliptic_txs_features[filter & (elliptic_txs_features[1] == i)]
    classes = elliptic_txs_classes[filter & (elliptic_txs_features[1] == i)]
    index_dt = classes.reset_index()['txId'].T.to_dict()
    index_dt = {v: k for k, v in index_dt.items()}
    edgelist = elliptic_txs_edgelist[(elliptic_txs_edgelist['graph'] == i) & (elliptic_txs_edgelist['classv1'] != 'unknown') & (elliptic_txs_edgelist['classv2'] != 'unknown')]
    edgelist = edgelist.apply(lambda row: updateVerticeNumber(row, index_dt), axis=1)[['txId1', 'txId2']]
    classes = classes.astype({'class': int})
    classes['class'] = classes['class'].apply(lambda x: x-1)
    
    adjacency_matrix = np.zeros((len(classes), len(classes)))
    for i in range(len(edgelist)):
        adjacency_matrix[edgelist.reset_index().loc[i].txId1][edgelist.reset_index().loc[i].txId2] = 1
    
    adjacency_matrix = torch.tensor(adjacency_matrix)
    x = torch.tensor(features.drop(0, axis=1).values)
    y = torch.tensor(classes['class'].values)
    edgelist = torch.tensor(edgelist.T.values)
    datas.append(Data(x=x, y=y, edge_index=edgelist, adjacency_matrix=adjacency_matrix))
    

In [7]:
datas

[Data(x=[2147, 166], edge_index=[2, 1924], y=[2147], adjacency_matrix=[2147, 2147]),
 Data(x=[1117, 166], edge_index=[2, 858], y=[1117], adjacency_matrix=[1117, 1117]),
 Data(x=[1279, 166], edge_index=[2, 727], y=[1279], adjacency_matrix=[1279, 1279]),
 Data(x=[1440, 166], edge_index=[2, 1169], y=[1440], adjacency_matrix=[1440, 1440]),
 Data(x=[1882, 166], edge_index=[2, 1491], y=[1882], adjacency_matrix=[1882, 1882]),
 Data(x=[485, 166], edge_index=[2, 209], y=[485], adjacency_matrix=[485, 485]),
 Data(x=[1203, 166], edge_index=[2, 858], y=[1203], adjacency_matrix=[1203, 1203]),
 Data(x=[1165, 166], edge_index=[2, 1044], y=[1165], adjacency_matrix=[1165, 1165]),
 Data(x=[778, 166], edge_index=[2, 484], y=[778], adjacency_matrix=[778, 778]),
 Data(x=[972, 166], edge_index=[2, 538], y=[972], adjacency_matrix=[972, 972]),
 Data(x=[696, 166], edge_index=[2, 477], y=[696], adjacency_matrix=[696, 696]),
 Data(x=[506, 166], edge_index=[2, 446], y=[506], adjacency_matrix=[506, 506]),
 Data(x=

## Salvando os Dados

In [13]:
if not os.path.exists('elliptic_pt'):
    os.mkdir('elliptic_pt')
    os.mkdir('elliptic_pt/train')
    os.mkdir('elliptic_pt/test')

In [14]:
for i in range(0,34):
    torch.save(datas[i], 'elliptic_pt/train/' + str(i + 1) + '.pt')
for i in range(34,49):
    torch.save(datas[i], 'elliptic_pt/test/' + str(i + 1) + '.pt')